# Tech Challenge DMCard

Nome: Mateus Vallim Martins

e-mail: mateus.vallim@usp.br

## Introdução

Para este desafio realizarei uma análise de todos os parênteses das expressões e não levarei em conta o restante de cada expressão, ou seja:
- Não verificarei se há operadores depois da abertura de parênteses, por exemplo "5-(+a/7)*15";
- Não verificarei se há operadores antes do fechamento de parênteses, por exemplo "5-(a/7+)*15";
- E não verificarei a falta de um operador antes da abertura ou depois do fechamento de parênteses, por exemplo "5(a/7)", o que na matemática indicaria uma multiplicação implícita

Antes de iniciar a execução do desafio vou analisar quantas vezes os parênteses aparecem em meio aos exemplos dados pelo próprio desafio, para que possamos ter um número de aparições razoável.

In [44]:
exemplos = ['a+(bc)-2-a','(a+b(2-c)-2+a)*2','(ab-(2+c)','2(3-a))',')3+b*(2-c)(']
todos_exemplos = ''.join(exemplos)

abrir = todos_exemplos.count('(')
fechar = todos_exemplos.count(')')
tudo = len(todos_exemplos)
print('Parenteses:',abrir+fechar)
print('Caracteres:',tudo)
print(round((abrir+fechar)*100/tudo),'%')

Parenteses: 16
Caracteres: 53
30 %


## Primeiros passos

Num primeiro momento vou criar a entrada com N expressões (1 <= N <= 10000), cada um com até 1000 caracteres, todos em formato de string tal que 30% destes caracteres serão caracteres de parênteses e os outros 70% será composto pelo alfabeto minúsculo, números e os operadores de soma "+", subtração "-", multiplicação "*" e divisão "/".

In [47]:
%%time

import pandas as pd
import random
import string  
import secrets
import re

# Criação do banco de dados de entrada
"""
serie = pd.Series('', name="entrada") # Uma nova série/coluna é criada para representar as equações de entrada

N = 10000 # Número da lista de expressões
for expressao in range(N): # É realizado um loop com N repetições para calcular uma expressão
    C = round(random.uniform(1,1000)) # a variável C representa o número de caracteres da expressão N, que pode ser de 1 até 1000
    e = '' # a variável e representa a expressão N em formato de string, começando sempre por uma string vazia
    for caracter in range(C): # Um novo loop é realizado para decidir cada caracter da expressão e
        if(random.random() > 0.3): # Como comentado, em 30% dos caracteres deverá ser parenteses
            if(random.random() > 0.5): # Metade dos 70% dos outros caracteres poderá ser dividida entre números ou letras minúsculas
                caract = secrets.choice(string.ascii_lowercase + string.digits)
            else: # E a outra metade será composta por operadores matemáticos
                caract = secrets.choice("+-*/")
        else: # Aqui são inseridos os 30% de caracteres de parenteses
            caract = secrets.choice("()")
        e = e+caract # Ao final de cada loop interno da expressão, a variável e é atualizada com o novo caracter decidido
    serie.loc[expressao] = e # Ao final de cada loop externo, a série serie é atualizada com a nova expressão e

serie.to_pickle("serie.pkl.compress", compression="gzip") # Salvar os dados de entrada
"""

serie = pd.read_pickle("serie.pkl.compress", compression="gzip") # Abrir os dados salvos

display(serie)

0       t*)3-2*-(u+-w+3*8lh*+-l+**(p/)-4a*((v9(5/5(-y+...
1       )-((3+(mwo*/(+sj6(*/-(*-zc(0*b(g++eyzi6++)(9(*...
2       w7yx-w/(/d/a))/f2-+)--)**(0-)*(/(i)(58/9o()k(-...
3       21)x+s-xtf/((t)(*u*-3)m)))()p)*)(*)-()*(+)+3/z...
4       (+9((()*)v2r**)+-/)(*+n)(+(xu)j4/*()()0p()+-()...
                              ...                        
9995    )(/s(90x-)*((*()o(j((*-+)//-+-*l)(/0+)0)(q--s-...
9996    (+)*5x/p)61++2)h-*/l/)q-/(()+-)(r)-r-2+/)5/i))...
9997    l/-+h-)/*(y/)x/(//(*(5*)(+-t*-r)()(+th)e(*a)b-...
9998    bzw*)+li)+))ca+*(-*e*)*6i/)/bs)*+8(x/)s+4)-/jo...
9999    )*pkw5(/)(o++c-1)/4/q*k(/-/(-)u)+(/)7/)b-)(++i...
Name: entrada, Length: 10000, dtype: object

Wall time: 67.8 ms


## Análise dos dados de entrada

Agora que temos uma coluna de dados (serie) com N expressões de quantidades diferentes de caracteres, faremos a análise de cada uma e isto resultará uma nova coluna de dados de strings 'incorrect' ou 'correct' correspondentes aos dados de entrada.

Para isto, primeiramente será retirado qualquer caracter que não seja parênteses de todas as expressões, assim é possível contar com mais facilidade o número de abertura e fechamento dos parênteses como mostrado abaixo.

(*a+3(/vn)-6/u**/hw4p*(a0q5*)+(+la)+6(c+()(**+... => (()()()(()(...

Com a expressão mais "limpa" podemos definir algumas características para uma expressão possivelmente correta:
- O número de '(' deve ser igual ao número de ')';
- Toda expressão deve iniciar com '(';
- E toda expressão que não conter parenteses está correta.

Após verificar as características acima é feita uma revisão na expressão, caracter por caracter, com uma contagem em que pra cada '(' é somado 1 e pra cada ')' é subtraído 1, tal que esta contagem demonstra a quantidade de parênteses em aberto do começo da expressão até algum caracter. Portanto, podemos inferir algumas possibilidades:
- Se esta contagem for menor do que 0, então existem ')' a mais até aquele ponto da expressão;
- Se esta contagem for maior do que 0 no último caracter, então existem '(' a mais em toda a expressão;
- Se esta contagem for igual a 0 no último caracter, então esta é uma expressão correta.

In [48]:
%%time

result = pd.Series('', name="saída") # Uma nova série/coluna é criada para representar os resultados da saída

for i in range(len(serie)): # É realizado um loop para analisar cada expressão
    S = serie.loc[i] # S é a expressão i da variável serie
    parentes = re.sub('\w|[+|\-|*|/]','',S) # parentes é uma string contendo somente os parênteses de S
    if(parentes.count('(') + parentes.count(')') == 0): # Se não existem parênteses, então é uma expressão correta
        result.loc[i] = 'correct'
    elif((parentes.count('(') == parentes.count(')')) and (parentes[0] == '(')): # Se o número de '(' for igual ao número de ')' e a expressão começar com '(', então ela é uma possível expressão correta
        parentes_count = 0 # parentes_count é a variável de contagem para os parênteses
        for C in range(len(parentes)): # É adidionado mais um loop com o número de repetições igual ao número de caracteres de parentes
            if(parentes[C] == '('): # Se o caracter da vez for um '(', então é adicionado 1 à contagem
                parentes_count = parentes_count + 1
            else: # Se o caracter da vez for um ')', então é subtraído 1 à contagem
                parentes_count = parentes_count - 1
            if(parentes_count < 0): # Se a contagem for menor do que 0 em qualquer momento, então quer dizer que a expressão é incorreta
                result.loc[i] = 'incorrect'
                continue # E é passada para a próxima expressão
            if(C == len(parentes) - 1): # Se o loop chegar até o último caracter de parentes podemos julgar se a expressão é correta ou não
                if(parentes_count == 0): # Se a contagem for igual a 0, então é uma expressão correta
                    result.loc[i] = 'correct'
                else: # Se a contagem for maior que 0, então é uma expressão incorreta
                    result.loc[i] = 'incorrect'
    else: # Se a contagem de '(' e ')' forem diferentes e alguma delas for maior que 0 ou parentes começar com ')', então a expressão é incorreta
        result.loc[i] = 'incorrect'

result.to_pickle("result.pkl.compress", compression="gzip") # Salvar os dados de saída

display(result)

0       incorrect
1       incorrect
2       incorrect
3       incorrect
4       incorrect
          ...    
9995    incorrect
9996    incorrect
9997    incorrect
9998    incorrect
9999    incorrect
Name: saída, Length: 10000, dtype: object

Wall time: 8.14 s


## Resultados

Nos resultados podemos analisar se os dois têm o mesmo tamanho e ver quantos são corretos e quanto são incorretos.

In [58]:
qtde = result.value_counts()

print('serie é igual a result?',serie.shape == result.shape)
print('-----')
display(qtde)

serie é igual a result? True
-----


incorrect    9797
correct       203
Name: saída, dtype: int64